In [1]:
import sys
import argparse
import os
import pandas as pd
import numpy as np
import re
import math

from collections import Counter
#pathPrefix = '/Users/friedman/Desktop/mnt'
#sys.path.append(pathPrefix + '/ifs/work/taylorlab/friedman/myUtils')
#import analysis_utils 
#import mutationSigUtils

In [5]:
def summarize_n_hypermutated_and_signatures_of_cases(mutClassificationDir = '/Users/friedman/Desktop/hypermutationStatusIds/'):
    nCasesSummaryL = []
    signaturesSummaryL = []
    for f in os.listdir(mutClassificationDir):
        cancerType = re.sub('.tsv', '', f)
        filePath = os.path.join(mutClassificationDir, f)
        df = pd.read_table(filePath)
        hypermutatedDf = df[df['hypermutantClassification'] == 'Hypermutated']
        highMutBurdenDf = df[df['hypermutantClassification'] == 'highMutationBurden']
        
        nCasesSummaryL.append({'cancerType': cancerType,
        'nHypermutated': hypermutatedDf.shape[0], 'nTotal': df.shape[0], 'nHighMutBurden': highMutBurdenDf.shape[0]})
    
        #TODO make the code for plotting the signatures
        signatureCounter = Counter(hypermutatedDf['dominantSignature'])
        for signature, count in signatureCounter.items():
            signature = re.sub('mean_', '', signature)
            signaturesSummaryL.append({'cancerType': cancerType,
                'signature': signature, 'nCases': count, 
                'nHypermutatedCases': hypermutatedDf.shape[0], 'nTotal': df.shape[0]})
        
    casesSummaryDf = pd.DataFrame(nCasesSummaryL)
    signaturesSummaryDf = pd.DataFrame(signaturesSummaryL)
    return casesSummaryDf, signaturesSummaryDf
        

In [6]:
casesSummary, signaturesSummary = summarize_n_hypermutated_and_signatures_of_cases()

In [12]:
#ADJUST THE CASES SUMMARY INFORMATION
minNCasesToDisplay = 250 #lump all cancers with fewer than this number of cases as other
casesSummary['cancerType'] = casesSummary.apply(lambda row: 'other' if row['nTotal'] < minNCasesToDisplay else row['cancerType'] ,axis=1)


In [14]:
#ADJUST THE SIGNATURES SUMMARY INFORMATION
minNHypermutatedCasesToDisplay = 10
signaturesSummary['cancerType'] = signaturesSummary.apply(lambda row: 'other' if row['nHypermutatedCases'] < minNHypermutatedCasesToDisplay else row['cancerType'] ,axis=1)
signaturesSummary['orderingVal'] = signaturesSummary.apply(lambda row: -1 if row['cancerType'] == 'other' else row['nHypermutatedCases'], axis=1)

signaturesRenameDict = {'1': 'MMR', 'SMOKING': 'SMOKING',
                        'MMR': 'MMR', 'APOBEC': 'APOBEC', '10': 'POLE',
                       '11': 'TMZ', '14': 'POLE&MMR', '7': 'UV'}
signaturesSummary['signature'] = signaturesSummary['signature'].apply(lambda x:
        signaturesRenameDict[x] if x in signaturesRenameDict else 'other')

In [15]:
signaturesSummary.to_csv('/Users/friedman/Desktop/WORK/dataForLocalPlotting/figure1cSignatureSummary.tsv', index=False, sep='\t')
casesSummary.to_csv('/Users/friedman/Desktop/WORK/dataForLocalPlotting/figure1bCancerTypeSummary.tsv', index=False, sep='\t')

In [ ]:
def get_per_case_nmut_mb_info(nmutDfPath = pathPrefix + '/ifs/work/taylorlab/friedman/hypermutationAnalysisProj/projectDataAndConfigFiles/nmutInfo_impact_filtered.tsv'):
    df = pd.read_table(nmutDfPath)
    return dict(zip(df['Tumor_Sample_Barcode'], df['Nmut_Mb']))

In [ ]:
allImpactMuts = analysis_utils.load_in_df_with_progress(pathPrefix + '/ifs/work/taylorlab/friedman/hypermutationAnalysisProj/projectDataAndConfigFiles/all_impact_mutations_annotated_cohort.maf', 275000)

In [14]:
hypermutantIdDir=pathPrefix +'/ifs/work/taylorlab/friedman/hypermutationAnalysisProj/projectDataAndConfigFiles/hypermutationStatusIds'
exceptionCancerTypes = set(['Bladder Cancer', 'Melanoma'])
listOfDfs = []
for f in os.listdir(hypermutantIdDir):
    cancerType = re.sub('_', ' ', f.strip('.tsv'))
    df = pd.read_table(hypermutantIdDir + '/' + f)
    dfHypermutators = df[df['hypermutantClassification'] == 'Hypermutated']
    dfHypermutators['cancerType'] = cancerType
    listOfDfs.append(dfHypermutators)
    
concatDf = pd.concat(listOfDfs)


/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


**add signature information**

In [41]:
impactSigs = pd.read_table(pathPrefix + '/ifs/work/taylorlab/friedman/hypermutationAnalysisProj/projectDataAndConfigFiles/signatures_from_unfiltered_maf.txt')

In [42]:
impactSigs = mutationSigUtils.merge_signature_columns(impactSigs)
impactSigs['dominantSignature'] = impactSigs.apply(lambda row: 
        mutationSigUtils.get_dominant_signature(row.to_dict(), cols=None, prefix='mean', notEnoughMuts= True), axis=1)

In [43]:
dominantSignatureDict = dict(zip(impactSigs['Tumor_Sample_Barcode'], impactSigs['dominantSignature']))

In [ ]:
finalDf['dominantSignature'] = finalDf['Tumor_Sample_Barcode'].apply(lambda x: dominantSignatureDict[x] if x in dominantSignatureDict else None) 

In [ ]:
finalDf['dominantSignatureDisplay'] = finalDf['dominantSignature'].apply(lambda x: 
                                                                         'MMR' if x in set(['mean_MMR', 'mean_1'])
                                                                         else 'POLE' if x == 'mean_10'
                                                                         else 'TMZ' if x == 'mean_11'
                                                                         else 'POLE+MMR' if x == 'mean_14'
                                                                         else 'SMOKING' if x == 'mean_4'
                                                                         else 'UV' if x == 'mean_7'
                                                                         else 'APOBEC' if x == 'mean_APOBEC'
                                                                         else 'Other')

**make it a long format df**

In [61]:
finalMeltedDf = pd.melt(finalDf, value_vars=['dominantSignatureDisplay'], id_vars=['Nmut_Mb', 'Tumor_Sample_Barcode', 'cancerType'])

In [82]:
orderingValDict = {'POLE':-1000, 'POLE+MMR':-2000, 'MMR':-3000, 'TMZ':-4000,
                   'UV':-5000, 'APOBEC':-6000, 'SMOKING':-7000, 'Other':-8000}
finalMeltedDf['orderingVal'] = finalMeltedDf.apply(lambda row: orderingValDict[row['value']] + row['Nmut_Mb'], axis=1)


In [84]:
finalMeltedDf.to_csv('/Users/friedman/Desktop/WORK/dataForLocalPlotting/cohortDisplayFigure.tsv', index=False, sep='\t')

In [90]:
len('Cancer of Unknown Primary')

25